In [51]:
import pandas as pd
import numpy as np
import time
import os

from sklearn.cluster import KMeans
from functools import cmp_to_key
import sys

INPUT = "datasets"
OUTPUT = "part1_results"
if not os.path.exists(OUTPUT):
    os.makedirs(OUTPUT)

SEED = 42

# Import datasets

## Preprocessing Methods

In [52]:
# preprocessing method

# Change categorical data to numerical data
def cat_2_num(df:pd.DataFrame):
    cat_columns = df.select_dtypes(['object']).columns
    df[cat_columns] = df[cat_columns].astype('category')
    df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
    return df

# Use KMeans to get discretized data
def get_discretization_data(df:pd.DataFrame):
    for col_name in df.columns:
        if(len(pd.unique(df[col_name])) <= 5):
            return df
        k = 5
        k_model = KMeans(n_clusters=k)
        k_model.fit(df[col_name].values.reshape(len(df[col_name]), 1))
        c = pd.DataFrame(k_model.cluster_centers_, columns = list("a")).sort_values(by = "a")
        w = c.rolling(2).mean().iloc[1:]
        w = np.asarray(w.values)
        w = [i[0] for i in w]
        w = [0] + w + [df[col_name].max()]
        df[col_name] = pd.cut(df[col_name], w, labels = range(k))
    return df

## UCI datasets

In [53]:
#German dataset
def German():
    df = pd.read_table(os.path.join(INPUT,"german.data-numeric"),delim_whitespace = True, header = None)
    df = get_discretization_data(df)
    df[24] = df[24]-1    # change label from 1,2 to 0,1
    return df

In [54]:
def Australian():
    df = pd.read_table(os.path.join(INPUT,"australian.dat"),delim_whitespace = True, header = None)
    df = get_discretization_data(df)
    return df

In [55]:
def Crx():
    df = pd.read_csv(os.path.join(INPUT,"crx.data"), header = None)
    # drop entries with ?
    df = df.replace("?", np.nan).dropna()
    # convert category data to numerical data
    df = cat_2_num(df)
    df = get_discretization_data(df)
    return df

In [56]:
def Hepatitis():
    df = pd.read_csv(os.path.join(INPUT,"hepatitis.data"), header = None)
    df = cat_2_num(df)
    df = get_discretization_data(df)
    df[19] = df[19]-1 # change to 0 or 1
    return df

In [57]:
def Ionosphere():
    df = pd.read_csv(os.path.join(INPUT, "ionosphere.data"), header=None)
    df = cat_2_num(df)
    df = get_discretization_data(df)
    return df

## Additional Kaggle datasets

In [58]:
def Pumpkin():
    df = pd.read_excel(os.path.join("datasets",'Pumpkin_Seeds_Dataset.xlsx'), sheet_name='Pumpkin_Seeds_Dataset',engine='openpyxl')
    df = cat_2_num(df)
    df = get_discretization_data(df)
    return df

In [59]:
# 5644 samples, relatively large dataset
def Mushroom():
    df = pd.read_csv(os.path.join(INPUT,'mushrooms.csv'))
    df = df.replace("?", np.nan).dropna()
    df = cat_2_num(df)
    df = get_discretization_data(df)
    return df

In [60]:
def Diabetes():
    df = pd.read_csv(os.path.join(INPUT,'diabetes_data.csv'), sep=';')
    df = cat_2_num(df)
    df = get_discretization_data(df)
    return df

# Classification

In [61]:
class RuleItem:
    """
    cond_set: a dict with following fashion:
            {item name: value, item name: value, ...}
        e.g.
            {A: 1, B: 1} (A, B are name of columns, here called "item", and in our code should be numerical index
                          but not string)
    class_label: just to identify the class it belongs to.
    dataset: a list returned by read method. (see read.py)
    cond_sup_count, rule_sup_count, support and confidence are number.
    """
    def __init__(self, cond_set, class_label, dataset):
        self.cond_set = cond_set
        self.class_label = class_label
        self.cond_sup_count, self.rule_sup_count = self._get_sup_count(dataset)
        self.support = self._get_support(len(dataset))
        self.confidence = self._get_confidence()

    # calculate condsupCount and rulesupCount
    def _get_sup_count(self, dataset):
        cond_sup_count = 0
        rule_sup_count = 0
        for case in dataset:
            is_contained = True
            for index in self.cond_set:
                if self.cond_set[index] != case[index]:
                    is_contained = False
                    break
            if is_contained:
                cond_sup_count += 1
                if self.class_label == case[-1]:
                    rule_sup_count += 1
        return cond_sup_count, rule_sup_count

    # calculate support count
    def _get_support(self, dataset_size):
        return self.rule_sup_count / dataset_size

    # calculate confidence
    def _get_confidence(self):
        if self.cond_sup_count != 0:
            return self.rule_sup_count / self.cond_sup_count
        else:
            return 0

    # print out the ruleitem
    def print(self):
        cond_set_output = ''
        for item in self.cond_set:
            cond_set_output += '(' + str(item) + ', ' + str(self.cond_set[item]) + '), '
        cond_set_output = cond_set_output[:-2]
        print('<({' + cond_set_output + '}, ' + str(self.cond_sup_count) + '), (' +
              '(class, ' + str(self.class_label) + '), ' + str(self.rule_sup_count) + ')>')

    # print out rule
    def print_rule(self):
        cond_set_output = ''
        for item in self.cond_set:
            cond_set_output += '(' + str(item) + ', ' + str(self.cond_set[item]) + '), '
        cond_set_output = '{' + cond_set_output[:-2] + '}'
        print(cond_set_output + ' -> (class, ' + str(self.class_label) + ')')

In [62]:
class FrequentRuleitems:
    """
    A set of frequent k-ruleitems, just using set.
    """
    def __init__(self):
        self.frequent_ruleitems_set = set()

    # get size of set
    def get_size(self):
        return len(self.frequent_ruleitems_set)

    # add a new ruleitem into set
    def add(self, rule_item):
        is_existed = False
        for item in self.frequent_ruleitems_set:
            if item.class_label == rule_item.class_label:
                if item.cond_set == rule_item.cond_set:
                    is_existed = True
                    break
        if not is_existed:
            self.frequent_ruleitems_set.add(rule_item)

    # append set of ruleitems
    def append(self, sets):
        for item in sets.frequent_ruleitems:
            self.add(item)

    # print out all frequent ruleitems
    def print(self):
        for item in self.frequent_ruleitems_set:
            item.print()


class Car:
    """
    Class Association Rules (Car). If some ruleitems has the same condset, the ruleitem with the highest confidence is
    chosen as the Possible Rule (PR). If there're more than one ruleitem with the same highest confidence, we randomly
    select one ruleitem.
    """
    def __init__(self):
        self.rules = set()
        self.pruned_rules = set()

    # print out all rules
    def print_rule(self):
        for item in self.rules:
            item.print_rule()

    # print out all pruned rules
    def print_pruned_rule(self):
        for item in self.pruned_rules:
            item.print_rule()

    # add a new rule (frequent & accurate), save the ruleitem with the highest confidence when having the same condset
    def _add(self, rule_item, minsup, minconf):
        if rule_item.support >= minsup and rule_item.confidence >= minconf:
            if rule_item in self.rules:
                return
            for item in self.rules:
                if item.cond_set == rule_item.cond_set and item.confidence < rule_item.confidence:
                    self.rules.remove(item)
                    self.rules.add(rule_item)
                    return
                elif item.cond_set == rule_item.cond_set and item.confidence >= rule_item.confidence:
                    return
            self.rules.add(rule_item)

    # convert frequent ruleitems into car
    def gen_rules(self, frequent_ruleitems, minsup, minconf):
        for item in frequent_ruleitems.frequent_ruleitems_set:
            self._add(item, minsup, minconf)

    # prune rules
    def prune_rules(self, dataset):
        for rule in self.rules:
            pruned_rule = prune(rule, dataset)

            is_existed = False
            for rule in self.pruned_rules:
                if rule.class_label == pruned_rule.class_label:
                    if rule.cond_set == pruned_rule.cond_set:
                        is_existed = True
                        break

            if not is_existed:
                self.pruned_rules.add(pruned_rule)

    # union new car into rules list
    def append(self, car, minsup, minconf):
        for item in car.rules:
            self._add(item, minsup, minconf)


# try to prune rule
def prune(rule, dataset):
    import sys
    min_rule_error = sys.maxsize
    pruned_rule = rule

    # prune rule recursively
    def find_prune_rule(this_rule):
        nonlocal min_rule_error
        nonlocal pruned_rule

        # calculate how many errors the rule r make in the dataset
        def errors_of_rule(r):

            errors_number = 0
            for case in dataset:
                if is_satisfy(case, r) == False:
                    errors_number += 1
            return errors_number

        rule_error = errors_of_rule(this_rule)
        if rule_error < min_rule_error:
            min_rule_error = rule_error
            pruned_rule = this_rule
        this_rule_cond_set = list(this_rule.cond_set)
        if len(this_rule_cond_set) >= 2:
            for attribute in this_rule_cond_set:
                temp_cond_set = dict(this_rule.cond_set)
                temp_cond_set.pop(attribute)
                temp_rule = RuleItem(temp_cond_set, this_rule.class_label, dataset)
                temp_rule_error = errors_of_rule(temp_rule)
                if temp_rule_error <= min_rule_error:
                    min_rule_error = temp_rule_error
                    pruned_rule = temp_rule
                    if len(temp_cond_set) >= 2:
                        find_prune_rule(temp_rule)

    find_prune_rule(rule)
    return pruned_rule


# invoked by candidate_gen, join two items to generate candidate
def join(item1, item2, dataset):
    if item1.class_label != item2.class_label:
        return None
    category1 = set(item1.cond_set)
    category2 = set(item2.cond_set)
    if category1 == category2:
        return None
    intersect = category1 & category2
    for item in intersect:
        if item1.cond_set[item] != item2.cond_set[item]:
            return None
    category = category1 | category2
    new_cond_set = dict()
    for item in category:
        if item in category1:
            new_cond_set[item] = item1.cond_set[item]
        else:
            new_cond_set[item] = item2.cond_set[item]
    new_ruleitem = RuleItem(new_cond_set, item1.class_label, dataset)
    return new_ruleitem


# similar to Apriori-gen in algorithm Apriori
def candidate_gen(frequent_ruleitems, dataset):
    returned_frequent_ruleitems = FrequentRuleitems()
    for item1 in frequent_ruleitems.frequent_ruleitems_set:
        for item2 in frequent_ruleitems.frequent_ruleitems_set:
            new_ruleitem = join(item1, item2, dataset)
            if new_ruleitem:
                returned_frequent_ruleitems.add(new_ruleitem)
                if returned_frequent_ruleitems.get_size() >= 1000:      # not allow to store more than 1000 ruleitems
                    return returned_frequent_ruleitems
    return returned_frequent_ruleitems


# main method, implementation of CBA-RG algorithm
def rule_generator(dataset, minsup, minconf):
    frequent_ruleitems = FrequentRuleitems()
    car = Car()

    # get large 1-ruleitems and generate rules
    class_label = set([x[-1] for x in dataset])
    for column in range(0, len(dataset[0])-1):
        distinct_value = set([x[column] for x in dataset])
        for value in distinct_value:
            cond_set = {column: value}
            for classes in class_label:
                rule_item = RuleItem(cond_set, classes, dataset)
                if rule_item.support >= minsup:
                    frequent_ruleitems.add(rule_item)
    car.gen_rules(frequent_ruleitems, minsup, minconf)
    cars = car

    last_cars_number = 0
    current_cars_number = len(cars.rules)
    while frequent_ruleitems.get_size() > 0 and current_cars_number <= 2000 and \
                    (current_cars_number - last_cars_number) >= 10:
        candidate = candidate_gen(frequent_ruleitems, dataset)
        frequent_ruleitems = FrequentRuleitems()
        car = Car()
        for item in candidate.frequent_ruleitems_set:
            if item.support >= minsup:
                frequent_ruleitems.add(item)
        car.gen_rules(frequent_ruleitems, minsup, minconf)
        cars.append(car, minsup, minconf)
        last_cars_number = current_cars_number
        current_cars_number = len(cars.rules)

    return cars

In [63]:
class CAR:
    def __init__(self):
        self.rules = set()
        self.pruned_rules = set()

    def print_rule(self):
        for rule in self.rules:
            rule.print()
    
    def print_pruned_rules(self):
        for rule in self.pruned_rules:
            rule.print()

    def _add(self, new_rule, minsup, minconf):
        if new_rule.support >= minsup and new_rule.confidence >= minconf:
            if new_rule in self.rules:
                return
            for rule in self.rules:
                if rule.cond_set == new_rule.cond_set:
                    if rule.confidence < new_rule.confidence:
                        self.rules.remove(rule)
                        self.rules.add(new_rule)
                    return
            self.rules.add(new_rule)
    
    def gen_rules(self, frequent_rules, minsup, minconf):
        for rules in frequent_rules.rule_dict.values():
            for rule in rules:
                self._add(rule, minsup, minconf)

    def prune_rules(self, dataset):
        for rule in self.rules:
            pruned_rule = prune(rule, dataset)

            existed = False
            for rule in self.pruned_rules:
                if rule.label == pruned_rule.label:
                    if rule.cond_set == pruned_rule.cond_set:
                        existed = True
                        break
            
            if not existed:
                self.prune_rules.add(pruned_rule)
    
    def append(self, car, minsup, minconf):
        for rule in car.rules:
            self._add(rule, minsup, minconf)

In [64]:
def is_satisfy(datacase, rule):
    for item in rule.cond_set:
        if datacase[item] != rule.cond_set[item]:
            return None
    if datacase[-1] == rule.class_label:
        return True
    else:
        return False


class Classifier:
    """
    This class is our classifier. The rule_list and default_class are useful for outer code.
    """
    def __init__(self):
        self.rule_list = list()
        self.default_class = None
        self._error_list = list()
        self._default_class_list = list()

    # insert a rule into rule_list, then choose a default class, and calculate the errors (see line 8, 10 & 11)
    def insert(self, rule, dataset):
        self.rule_list.append(rule)             # insert r at the end of C
        self._select_default_class(dataset)     # select a default class for the current C
        self._compute_error(dataset)            # compute the total number of errors of C

    # select the majority class in the remaining data
    def _select_default_class(self, dataset):
        class_column = [x[-1] for x in dataset]
        class_label = set(class_column)
        max = 0
        current_default_class = None
        for label in class_label:
            if class_column.count(label) >= max:
                max = class_column.count(label)
                current_default_class = label
        self._default_class_list.append(current_default_class)

    # compute the sum of errors
    def _compute_error(self, dataset):
        if len(dataset) <= 0:
            self._error_list.append(sys.maxsize)
            return

        error_number = 0

        # the number of errors that have been made by all the selected rules in C
        for case in dataset:
            is_cover = False
            for rule in self.rule_list:
                if is_satisfy(case, rule):
                    is_cover = True
                    break
            if not is_cover:
                error_number += 1

        # the number of errors to be made by the default class in the training set
        class_column = [x[-1] for x in dataset]
        error_number += len(class_column) - class_column.count(self._default_class_list[-1])
        self._error_list.append(error_number)

    # see line 14 and 15, to get the final classifier
    def discard(self):
        # find the first rule p in C with the lowest total number of errors and drop all the rules after p in C
        index = self._error_list.index(min(self._error_list))
        self.rule_list = self.rule_list[:(index+1)]
        self._error_list = None

        # assign the default class associated with p to default_class
        self.default_class = self._default_class_list[index]
        self._default_class_list = None

    # just print out all selected rules and default class in our classifier
    def print(self):
        for rule in self.rule_list:
            rule.print_rule()
        print("default_class:", self.default_class)


# sort the set of generated rules car according to the relation ">", return the sorted rule list
def sort(car):
    def cmp_method(a, b):
        if a.confidence < b.confidence:     # 1. the confidence of ri > rj
            return 1
        elif a.confidence == b.confidence:
            if a.support < b.support:       # 2. their confidences are the same, but support of ri > rj
                return 1
            elif a.support == b.support:
                if len(a.cond_set) < len(b.cond_set):   # 3. both confidence & support are the same, ri earlier than rj
                    return -1
                elif len(a.cond_set) == len(b.cond_set):
                    return 0
                else:
                    return 1
            else:
                return -1
        else:
            return -1

    rule_list = list(car.rules)
    rule_list.sort(key=cmp_to_key(cmp_method))
    return rule_list


# main method of CBA-CB: M1
def classifier_builder_m1(cars, dataset):
    classifier = Classifier()
    cars_list = sort(cars)
    for rule in cars_list:
        temp = []
        mark = False
        for i in range(len(dataset)):
            is_satisfy_value = is_satisfy(dataset[i], rule)
            if is_satisfy_value is not None:
                temp.append(i)
                if is_satisfy_value:
                    mark = True
        if mark:
            temp_dataset = list(dataset)
            for index in temp:
                temp_dataset[index] = []
            while [] in temp_dataset:
                temp_dataset.remove([])
            dataset = temp_dataset
            classifier.insert(rule, dataset)
    classifier.discard()
    return classifier

In [65]:
# calculate the error rate of the classifier on the dataset
def get_error_rate(classifier, dataset):
    size = len(dataset)
    error_number = 0
    for case in dataset:
        is_satisfy_value = False
        for rule in classifier.rule_list:
            is_satisfy_value = is_satisfy(case, rule)
            if is_satisfy_value == True:
                break
        if is_satisfy_value == False:
            if classifier.default_class != case[-1]:
                error_number += 1
    return error_number / size


# 10-fold cross-validations on CBA (M1) without pruning
def cross_validate_m1_without_prune(dataset, minsup=0.01, minconf=0.5, quiet=False):

    block_size = int(len(dataset) / 10)
    split_point = [k * block_size for k in range(0, 10)]
    split_point.append(len(dataset))
    
    cba_rg_total_runtime = 0
    cba_cb_total_runtime = 0
    total_car_number = 0
    total_classifier_rule_num = 0
    error_total_rate = 0

    for k in range(len(split_point)-1):
        if not quiet:
            print("\nRound %d:" % k)

        training_dataset = dataset[:split_point[k]] + dataset[split_point[k+1]:]
        test_dataset = dataset[split_point[k]:split_point[k+1]]

        start_time = time.time()
        cars = rule_generator(training_dataset, minsup, minconf)
        end_time = time.time()
        cba_rg_runtime = end_time - start_time
        cba_rg_total_runtime += cba_rg_runtime

        start_time = time.time()
        classifier_m1 = classifier_builder_m1(cars, training_dataset)
        end_time = time.time()
        cba_cb_runtime = end_time - start_time
        cba_cb_total_runtime += cba_cb_runtime

        error_rate = get_error_rate(classifier_m1, test_dataset)
        error_total_rate += error_rate

        total_car_number += len(cars.rules)
        total_classifier_rule_num += len(classifier_m1.rule_list)

        if not quiet:
            print(f"CBA's error rate without pruning: {error_rate:.2%}")
            print(f"No. of CARs without pruning: {len(cars.rules)}")
            print(f"CBA-RG's run time without pruning: {cba_rg_runtime:.3f} s")
            print(f"CBA-CB M1's run time without pruning: {cba_cb_runtime:.3f} s")
            print(f"No. of rules in classifier of CBA-CB M1 without pruning: {len(classifier_m1.rule_list)}")

    if not quiet:    
        print(f"\nAverage CBA's error rate without pruning: {(error_total_rate / 10):.2%}")
        print(f"Average No. of CARs without pruning: {int(total_car_number / 10)}")
        print(f"Average CBA-RG's run time without pruning: {(cba_rg_total_runtime / 10):.3f} s")
        print(f"Average CBA-CB M1's run time without pruning: {(cba_cb_total_runtime / 10):.3f} s")
        print(f"Average No. of rules in classifier of CBA-CB M1 without pruning: {int(total_classifier_rule_num / 10)}")

    return error_total_rate / 10, int(total_car_number / 10), (cba_rg_total_runtime / 10), (cba_cb_total_runtime / 10), int(total_classifier_rule_num / 10)

In [66]:
dataset_funcs = [German, Australian, Crx, Hepatitis, Ionosphere, Pumpkin, Mushroom, Diabetes]
stats_list = {'dataset':[], 'error_rate':[], 'CBA_count':[], 'CBA_RG_runtime':[], 'CBA_CB_runtime':[], 'rule_count':[] }
for dataset_getter in dataset_funcs:
    print(f"Classifying {dataset_getter.__name__} dataset")
    avg_err_rate, avg_car_cnt, avg_rg_runtime, avg_cb_runtime, avg_rule_cnt = cross_validate_m1_without_prune(dataset_getter().values.tolist(), quiet=True)
    stats_list['dataset'].append(dataset_getter.__name__)
    stats_list['error_rate'].append(avg_err_rate)
    stats_list['CBA_count'].append(avg_car_cnt)
    stats_list['CBA_RG_runtime'].append(avg_rg_runtime)
    stats_list['CBA_CB_runtime'].append(avg_cb_runtime)
    stats_list['rule_count'].append(avg_rule_cnt)
pd.DataFrame(stats_list).to_csv(os.path.join(OUTPUT, 'CBA_results.csv'))

Classifying German dataset
Classifying Australian dataset
